## 문서 군집화 개요
문서 군집화는 비슷한 텍스트 구성의 문서를 군집화 하는 것. (**비지도 학습**)<br>
> 문서를 피처 벡터화 -> 군집화 알고리즘 적용(일반적으로 K-Means)<br>

<img src=https://blog.kakaocdn.net/dn/bHUXuA/btrF5HZxDAW/tvlOjgYhkIq5UKmnYauDQk/img.png width=1000>

## Opinion Review 데이터 세트 문서 군집화
uci 데이터 다운로드 링크
https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review

### 데이터 로드

In [93]:
import pandas as pd
import numpy as np
import glob ,os
import warnings
warnings.filterwarnings('ignore')

# 디렉토리 설정
path = '../data/OpinosisDataset1.0/topics'

# path로 지정한 디렉토리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합, 
# 개별 파일들의 파일내용은 DataFrame로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합 
for file_ in sorted(all_files):
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    
    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경. 
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
# 파일 내용 텍스트를 통으로 opinion_text에 값으로 할당
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


### **Lemmatization을 위한 함수 생성**
TfidfVectorizer의 tokenizer파라미터의 인자로 사용될 lemmatization 어근 변환 함수를 설정

In [94]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# punctuation: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' -> 모두 제거
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 입력으로 들어온 token단어들에 대해서 lemmatization 어근 변환.
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# TfidfVectorizer 객체 생성 시 tokenizer인자로 해당 함수를 설정하여 lemmatization 적용
# 입력으로 문장을 받아서 punctuation 제거 -> 소문자 변환 -> 단어 토큰화 -> lemmatization 어근 변환.
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
# 최종적으로 lemmatize된 토큰의 리스트를 반환

### **TF-IDF 피처 벡터화 (Lemmatization 적용)** 
**TF-IDF 피처 벡터화, TfidfVectorizer에서 피처 벡터화 수행 시 Lemmatization(어근 추출)을 적용하여 토큰화**
- Stemming과 Lemmatization 같은 어근변환은 TfidfVectorizer에서 직접 지원하진 않으나,<br>
 **tokenizer 파라미터**에 커스텀 어근 변환 함수를 적용하여 어근 변환을 수행할 수 있음
 <br>
 
- TfidfVectorizer 생성자의 tokenizer인자로 위에서 생성한 LemNorvalize 함수 설정.

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tokenizer에 위에서 정의한 LemNormalize함수 적용하고, stop_words 제거
# ngram 1,2까지 허용하고, 단어 최소 0.05이상 0.85이하 빈도있는 애들만
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

In [96]:
feature_vect.shape
# 51개 문서에 대해, 4611개의 단어(피처)들로 벡터화됨.

(51, 4611)

### **K-Means군집화**

#### **5개의 군집으로 K-Means 군집화**

In [97]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [98]:
print(f"51개 문서에 대해, 5개의 군집으로 K-Means 군집화된 결과\n{km_cluster.labels_}")
km_cluster.labels_.shape

51개 문서에 대해, 5개의 군집으로 K-Means 군집화된 결과
[2 0 1 1 1 2 4 4 2 2 2 2 2 3 3 3 4 4 4 1 3 3 4 2 3 4 1 3 3 4 0 0 0 2 2 2 2
 4 3 3 3 1 1 2 1 3 3 4 2 2 2]


(51,)

#### **데이터 별로 군집화 확인**

In [99]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
1,bathroom_bestwestern_hotel_sfo,...,0
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1


#### 군집별로 filename 확인

In [100]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')
# room 관련 군집인것으로 확인됨 (비지도니까 타겟값이 따로 없다.)

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,0
30,room_holiday_inn_london,...,0
31,rooms_bestwestern_hotel_sfo,...,0
32,rooms_swissotel_chicago,...,0


In [101]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')
# 배터리, 키보드, 넷북 등 전자기기 관련 군집으로 확인된다.

,filename,opinion_text,cluster_label
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
19,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


In [102]:
document_df[document_df['cluster_label']==2].sort_values(by='filename').head(5)
# kindle 도 꽤 있지만, 전반적으로 nuvi(네비종류)_gps가 대부분

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
5,buttons_amazon_kindle,...,2
8,directions_garmin_nuvi_255W_gps,...,2
9,display_garmin_nuvi_255W_gps,...,2
10,eyesight-issues_amazon_kindle,...,2


In [103]:
document_df[document_df['cluster_label']==3].sort_values(by='filename').head(7)
# 대부분 hotel, inn에 대한것. 여기서도 kindle이 좀 끼어들었네

,filename,opinion_text,cluster_label
13,food_holiday_inn_london,...,3
14,food_swissotel_chicago,...,3
15,free_bestwestern_hotel_sfo,...,3
20,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
24,parking_bestwestern_hotel_sfo,...,3
27,price_amazon_kindle,...,3


In [104]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')
# honda, toyota 차종 그리고 년도로 확실하게 클러스터링 됨.

,filename,opinion_text,cluster_label
6,comfort_honda_accord_2008,...,4
7,comfort_toyota_camry_2007,...,4
16,gas_mileage_toyota_camry_2007,...,4
17,interior_honda_accord_2008,...,4
18,interior_toyota_camry_2007,...,4
22,mileage_honda_accord_2008,...,4
25,performance_honda_accord_2008,...,4
29,quality_toyota_camry_2007,...,4
37,seats_honda_accord_2008,...,4
47,transmission_toyota_camry_2007,...,4


#### 3개의 군집으로 K-Means 군집화

In [105]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')
# 0은 전자기기, 네비(gps) kindle이 좀 있는걸로 군집화 되었고
# 1은 차종, 2는 hotel, inn으로 클러스터링 되었다.

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
48,updates_garmin_nuvi_255W_gps,...,0
44,speed_windows7,...,0
43,speed_garmin_nuvi_255W_gps,...,0
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
41,size_asus_netbook_1005ha,...,0
36,screen_netbook_1005ha,...,0
35,screen_ipod_nano_8gb,...,0
34,screen_garmin_nuvi_255W_gps,...,0
33,satellite_garmin_nuvi_255W_gps,...,0


### 군집(Cluster)별 핵심 단어 추출하기


In [106]:
feature_vect.shape

(51, 4611)

#### cluster_centers_ 의미
KMeans객체의 cluster_centers_ 속성은 개별 피처들의 클러스터 중심과의 상대 위치를 정규화된 숫자값으로 표시

**0~1까지의 값으로 표현되며 1에 가까울 수록 중심에 더 가깝다는 의미**

In [107]:
cluster_centers = km_cluster.cluster_centers_
print(f'cluster_centers shape :{cluster_centers.shape}\n')
print(f"cluster_centers: {cluster_centers.shape[0]}개의 클러스터에 대해,\
{cluster_centers.shape[1]}개 단어(피처)가 cluster_center에 얼마나 가까이 있는지 값을 갖는다.\n")
print(cluster_centers)

cluster_centers shape :(3, 4611)

cluster_centers: 3개의 클러스터에 대해,4611개 단어(피처)가 cluster_center에 얼마나 가까이 있는지 값을 갖는다.

[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


#### 군집별 핵심단어 등 추출 함수 생성
**군집별 top n 핵심단어, 그 단어의 중심 위치와의 거리값, 대상 파일명들을 반환하는 함수 생성**

In [108]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
# fit이 된 cluster_model이 들어감
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순(center와 가까운 순)으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num): # clusters_num=3 -> cluster_num: 0, 1, 2
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num # { 0: {'cluster': 0}, ...}
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        # top_feature_indexes는 인덱스의 배열
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features] # 해당 cluster행의, top_n_features개수만큼의 열
        # top_features는 top 10개 단어 리스트
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심과 거리값 배열
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

#### 군집별 핵심 단어 등 출력 함수 생성
**클러스터별 top feature들의 단어와 파일명 출력**

In [114]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print(f"Top features value: {np.round(cluster_detail['top_features_value'],2)}")
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================\n')

In [110]:
tfidf_vect.get_feature_names()[:10]

['0 5', '0 great', '0 room', '00', '05', '06', '07', '08', '1 2', '1 block']

In [111]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )

In [112]:
cluster_details

{0: {'cluster': 0,
  'top_features': ['screen',
   'battery',
   'keyboard',
   'battery life',
   'life',
   'kindle',
   'direction',
   'video',
   'size',
   'voice'],
  'top_features_value': [0.13738120192560713,
   0.12029588637881046,
   0.06416898221290011,
   0.06330587553655656,
   0.058609811414901014,
   0.05646823079053855,
   0.05393721234628231,
   0.05389363145594924,
   0.05127615400589759,
   0.05119379623996657],
  'filenames': ['accuracy_garmin_nuvi_255W_gps',
   'battery-life_amazon_kindle',
   'battery-life_ipod_nano_8gb',
   'battery-life_netbook_1005ha',
   'buttons_amazon_kindle',
   'directions_garmin_nuvi_255W_gps',
   'display_garmin_nuvi_255W_gps',
   'eyesight-issues_amazon_kindle',
   'features_windows7',
   'fonts_amazon_kindle',
   'keyboard_netbook_1005ha',
   'navigation_amazon_kindle',
   'performance_netbook_1005ha',
   'price_amazon_kindle',
   'satellite_garmin_nuvi_255W_gps',
   'screen_garmin_nuvi_255W_gps',
   'screen_ipod_nano_8gb',
   'screen

#### 출력 결과

In [115]:
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Top features value: [0.14 0.12 0.06 0.06 0.06 0.06 0.05 0.05 0.05 0.05]
Reviews 파일명 : ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']

####### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Top features value: [0.23 0.19 0.18 0.13 0.12 0.12 0.1  0.1  0.1  0.09]
Reviews 파일명 : ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'mileage_honda_accord_2008', 'performance_honda_accord_2008']

####### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'cle